# XFaster Tutorial Test

**A. Gambrel and A. Rahlin | Last updated: 4/21/21**
***

**There are three primary goals of this guide:**
1. To explain the math of how XFaster works.
2. To show how that math is implemented in the code.
3. To show how to use the code and interact with its outputs.

This guide will step through the XFaster algorithm showing the math and what it looks like in its code implementation. Because XFaster takes a while to process through all the maps it takes as input, this notebook is not intended to be run by individual users. 

Much of this guide is copied directly from the materials in [the XFaster wiki reference](http://spiderwiki.princeton.edu/wiki/ReferenceDocuments/XFaster).

## How we get to the equations we ultimately want to solve

XFaster is a hybrid of two types of estimators: a pseudo-$C_\ell$ Monte Carlo estimator a la MASTER and PolSPICE, and a quadratic estimator. The latter is the special sauce, so we'll go into some detail about it here.

First, we'll introduce the Newton method for iteratively finding the root of a function. There are helpful pictures to visualize how this method works [on wikipedia](https://en.wikipedia.org/wiki/Newton%27s_method). Say you have some continuous function, $f$, and your initial guess for its root is $x_0$. We can Taylor expand $f$ around this guess like so:

\begin{equation}
f\left(x_{0}+\epsilon\right)=f\left(x_{0}\right)+f^{\prime}\left(x_{0}\right) \epsilon+\frac{1}{2} f^{\prime \prime}\left(x_{0}\right) \epsilon^{2}+\ldots
\end{equation}

And to first order, 
\begin{equation}
f\left(x_{0}+\epsilon\right) \approx f\left(x_{0}\right)+f^{\prime}\left(x_{0}\right) \epsilon
\end{equation}

We set the left side to zero, and solve for the step size for the next iteration:

\begin{equation}
\epsilon_{0}=-\frac{f\left(x_{0}\right)}{f^{\prime}\left(x_{0}\right)}
\end{equation}

We solve for $f$ and $f'$ at the new guess, and continue the iteration, where the forumula for each next guess of the root is:

\begin{equation}
x_{n+1}=x_{n}+\epsilon_{n}=x_{n}-\frac{f\left(x_{n}\right)}{f^{\prime}\left(x_{n}\right)}
\end{equation}

A major caveat with using this method is that if the derivative goes to zero near the starting or final root guess, things blow up and it will not converge. However, if the function is well behaved, and the initial guess is close enough, it can be shown that this method converges quadratically.

Ultimately, XFaster isn't going to solve for a root; it's going to find the extreme of some function. So instead of finding where $f=0$, we want to find where its first derivative goes to 0. So now we're going to take the first derivative of equation 1 and set it equal to 0:

\begin{equation}
0=\frac{d}{dx}\left(f\left(x_{0}\right)+f^{\prime}\left(x_{0}\right) \epsilon_{0}+\frac{1}{2} f^{\prime \prime}\left(x_{0}\right) \epsilon_{0}^{2}\right)=f^{\prime}\left(x_{0}\right)+f^{\prime \prime}\left(x_{0}\right) \epsilon_{0}
\end{equation}

where we're throwing out terms higher than second derivative. Now, our step size is given by

\begin{equation}
\epsilon_{0}=-\frac{f^{\prime}\left(x_{0}\right)}{f^{\prime \prime}\left(x_{0}\right)}
\end{equation}

Extending this method to multiple variables, the first derivative becomes a gradient, and the second derivative becomes the Hessian:

\begin{equation}
\mathbf{H}=\left[\begin{array}{cccc}{\frac{\partial^{2} f}{\partial x_{1}^{2}}} & {\frac{\partial^{2} f}{\partial x_{1} \partial x_{2}}} & {\cdots} & {\frac{\partial^{2} f}{\partial x_{1} \partial x_{n}}} \\ {\frac{\partial^{2} f}{\partial x_{2} \partial x_{1}}} & {\frac{\partial^{2} f}{\partial x_{2}^{2}}} & {\cdots} & {\frac{\partial^{2} f}{\partial x_{2} \partial x_{n}}} \\ {\vdots} & {\vdots} & {\ddots} & {\vdots} \\ {\frac{\partial^{2} f}{\partial x_{n} \partial x_{1}}} & {\frac{\partial^{2} f}{\partial x_{n} \partial x_{2}}} & {\cdots} & {\frac{\partial^{2} f}{\partial x_{n}^{2}}}\end{array}\right]
\end{equation}

for an expression for the local extremum of 

\begin{equation}
x^1_{n}=x^0_{n}-\mathbf{H}^{-1} \nabla f\left(x^0_{n}\right)
\label{eq:step}
\end{equation}

Because it's costly to compute $H$ for each iteration, we can instead make the approximation of using its expectation value, which does not depend on the data. This is equivalent to the Fisher information matrix:

\begin{equation}
\mathcal{F}_{i j}=\left\langle\mathbf{H}_{i j}\right\rangle=\left\langle\frac{\partial^{2} f}{\partial x_{i} \partial x_{j}}\right\rangle
\label{eq:fish_approx}
\end{equation}

This has all so far been an abstract exercise in how to find the values of the variables that maximize some function that depends on them. Now let's get into what XFaster uses it for, maximizing the likelihood function, which we approximate to be Gaussian:

\begin{equation}
\mathcal{L}(\mathbf{d} | \theta)=\frac{1}{|2 \pi \mathbf{C}|^{1 / 2}} \exp \left(-\frac{1}{2} \mathbf{d} \cdot \mathbf{C}^{-1} \cdot \mathbf{d}^{T}\right)
\end{equation}

where $\mathbf{d}$ is an observed data set, $\theta$ are the model parameters, and $\mathbf{C}$ is the covariance matrix, which depends on the model parameters: $\textbf{C}(\theta)=\textbf{S}(\theta)+\textbf{N}$, where $\textbf{S}$ is signal and $\textbf{N}$ is noise.

For XFaster, our parameters, $\theta$ that will be fit to the data are the bandpowers, $\mathcal{C}_\ell$. We want to maximize the log likelihood (so we can take derivatives more easily and since it is maximized where the likelihood is maximized), so we can use Equation $\ref{eq:step}$ and the Fisher approximation of Equation $\ref{eq:fish_approx}$ to write down the size of the step we need from our initial bandpower guess:

\begin{equation}
\delta \mathcal{C}_{\ell}=\frac{1}{2} \sum_{\ell^\prime} \mathcal{F}_{\ell \ell^{\prime}}^{-1} \operatorname{Tr}\left[\left(\mathbf{C}^{-1} \frac{\partial \mathbf{S}}{\partial \mathcal{C}_{\ell}} \mathbf{C}^{-1}\right)\left(\mathbf{d} \mathbf{d}^{T}-\mathbf{C}\right)\right]
\label{eq:cell}
\end{equation}

\begin{equation}
\mathcal{F}_{\ell \ell^{\prime}}=\frac{1}{2} \operatorname{Tr}\left[\mathbf{C}^{-1} \frac{\partial \mathbf{S}}{\partial \mathcal{C}_{\ell}} \mathbf{C}^{-1} \frac{\partial \mathbf{S}}{\partial \mathcal{C}_{\ell^{\prime}}} \right]
\label{eq:fisher_ell}
\end{equation}

where I've left out all the math to get the first and second derivatives. **Note: I will use $\mathcal{C}$ for bandpowers and $C$ for covariance. Similarly, the Fisher matrix will be $\mathcal{F}$ and the transfer function will be $F$.** 

Now, instead of iterating on the steps toward the maximum, XFaster iterates towards the bandpowers themselves. It does this by reconfiguring the second term in the trace in Equation $\ref{eq:cell}$, which should iteratively get closer to zero, and instead reformats it to be the estimate of the measured signal:

\begin{equation}
\mathcal{C}_{\ell}=\frac{1}{2} \sum_{\ell'} \mathcal{F}_{\ell \ell^{\prime}}^{-1} \operatorname{Tr}\left[\left(\mathbf{C_{\ell'}}^{-1} \frac{\partial \mathbf{S_{\ell'}}}{\partial \mathcal{C}_{\ell'}} \mathbf{C_{\ell'}}^{-1}\right)\left(\mathbf{C}_{\ell'}^{o b s}-\langle\mathbf{N_{\ell'}}\rangle\right)\right]
\end{equation}

where the $\langle\mathbf{N}\rangle\$ is the ensemble average of the noise simulations, needed to debias the total covariance of the data to leave an estimate of signal alone.

From here, XFaster makes a few more approximations to make the matrix operations manageable. We approximate our noise to be diagonal and uncorrelated with signal, and the signal will be averaged into bins to reduce correlations among modes from using less than the full sky. So now, the covariance for the cut sky is approximated as:

\begin{equation}
\tilde{C}_{\ell m, \ell^{\prime} m^{\prime}}=\delta_{\ell \ell^{\prime}} \delta_{m m^{\prime}}\left(\tilde{\mathcal{C}}_{\ell}+\left\langle\tilde{N}_{\ell}\right\rangle\right)
\end{equation}

The thing that our instrument measures is this pseudo-$\tilde{\mathcal{C}}_\ell$ spectrum. We ultimately want to know the full sky power spectrum, $\mathcal{C}_\ell$. For TT, for example, that's related to our measured $\tilde{\mathcal{C}}_\ell$s by

\begin{equation}
\tilde{\mathcal{C}}_{\ell}^{TT}=\sum_{\ell^{\prime}} K_{\ell \ell^{\prime}}^{TT} F_{\ell^{\prime}}^{TT} B_{\ell^{\prime}}^{2} \mathcal{C}_{\ell^{\prime}}^{TT}
\end{equation}

where  $K_{\ell, \ell'}$ is the coupling kernel that accounts for mode mixing due to the non-orthogonality of the spherical harmonic basis on the cut sky, $F_\ell$ is the filter transfer function, and $B_\ell$ is the beam window function.

This is written on an $\ell$ by $\ell$ basis, but in practice we'll want to bin to reduce signal correlations and increase signal to noise, so we add the binning operator $\chi_b$:

\begin{equation}
\tilde{\mathcal{C}}_{\ell}^{TT}=\sum_b q_b \sum_{\ell^{\prime}} K_{\ell \ell^{\prime}}^{TT} F_{\ell^{\prime}}^{TT} B_{\ell^{\prime}}^{2} \mathcal{C}_{\ell^{\prime}}^{TT} \chi_{b}\left(\ell^{\prime}\right)
\end{equation}

where I've now added in a coefficient, $q_b$, which accounts for any deviation of our measured bandpowers from the signal we expect our instrument to have measured. In practice, $q_b$ is actually what XFaster solves for. So now, instead of using $\mathcal{C}_\ell$ as the parameter we are optimizing, we instead solve for the maximum likelihood with respect to the bandpower deviations, $q_b$:

\begin{equation}
q_{b}=\frac{1}{2} \sum_{b^{\prime}} \mathcal{F}_{b b^{\prime}}^{-1} \sum_{\ell} (2 \ell+1) \operatorname{Tr}\left[ \left(\tilde{\mathbf{D}}_{\ell}^{-1} \frac{\partial \tilde{\mathbf{S}}_{\ell}}{\partial q_{b^{\prime}}} \tilde{\mathbf{D}}_{\ell}^{-1}\right)\mathbf{g}\left(\tilde{\mathbf{D}}_{\ell}^{o b s}-\tilde{\mathbf{N}}_{\ell}\right)\mathbf{g}^T\right]
\label{eq:qb}
\end{equation}

\begin{equation}
\mathcal{F}_{b b^{\prime}}=\frac{1}{2} \sum_{\ell} (2 \ell+1)\operatorname{Tr}\left[\tilde{\mathbf{D}}_{\ell}^{-1} \frac{\partial \tilde{\mathbf{S}}_{\ell}}{\partial q_{b}} \tilde{\mathbf{D}}_{\ell}^{-1} \mathbf{g}\frac{\partial \tilde{\mathbf{S}}_{\ell}}{\partial q_{b^{\prime}}}\mathbf{g}^T\right]
\label{eq:fisher}
\end{equation}

where now instead of solving for just TT for one map, I'm generalizing to a matrix form where 

\begin{equation}
\tilde{\mathbf{D}}_{\ell}=
\begin{bmatrix}
\tilde{\mathbf{D}}_{\ell}^{1x1} & \tilde{\mathbf{D}}_{\ell}^{1x2} & \tilde{\mathbf{D}}_{\ell}^{1x3} & \cdots & \tilde{\mathbf{D}}_{\ell}^{1xN} \\ 
\tilde{\mathbf{D}}_{\ell}^{2x1} & \tilde{\mathbf{D}}_{\ell}^{2x2} & \tilde{\mathbf{D}}_{\ell}^{2x3} & \cdots & \vdots \\ 
\tilde{\mathbf{D}}_{\ell}^{3x1} & \tilde{\mathbf{D}}_{\ell}^{3x2} & \tilde{\mathbf{D}}_{\ell}^{3x3} & \cdots & \vdots \\
\vdots & \vdots & \vdots & \ddots & \vdots \\
\tilde{\mathbf{D}}_{\ell}^{Nx1} & \cdots & \cdots & \cdots & \tilde{\mathbf{D}}_{\ell}^{NxN}\\
\end{bmatrix}
\label{eq:dell}
\end{equation}

where $N$ is the number of maps, and each element of the above matrix is a 3x3 subblock of $\tilde{C}_\ell$s for that map cross (*note: this the the full covariance, $\tilde{C}_\ell$, not only the signal part, $\tilde{\mathcal{C}}_\ell$*):
\begin{equation}
\tilde{\mathbf{D}}_{\ell}^{1\times 1}=\left[\begin{array}{ccc}{\tilde{\mathrm{C}}_{\ell}^{T T}} & {\tilde{\mathrm{C}}_{\ell}^{T E}} & {\tilde{\mathrm{C}}_{\ell}^{T B}} \\ {\tilde{\mathrm{C}}_{\ell}^{T E}} & {\tilde{\mathrm{C}}_{\ell}^{E E}} & {\tilde{\mathrm{C}}_{\ell}^{E B}} \\ {\tilde{\mathrm{C}}_{\ell}^{T B}} & {\tilde{\mathrm{C}}_{\ell}^{E B}} & {\tilde{\mathrm{C}}_{\ell}^{B B}}\end{array}\right]_{1\times 1}
\end{equation}

We've also reduced the trace over $\ell$ in equations \ref{eq:cell} and \ref{eq:fisher_ell} to the number of modes we measure, assuming isotropy: $\sum_{\ell}(2\ell+1)\mathbf{gg}^T$, where $g$ is a weighting factor accounting for the effective number of degrees of freedom of the map.  And the trace in equations \ref{eq:qb} and \ref{eq:fisher} is over the various map cross spectrum components.

There is some complication that arises from building the non-TT components of the signal covariance, which is that there is mixing between T$\leftrightarrow$E,B and E$\leftrightarrow$B caused by the masking. We account for this with the proper combination of shape operators, $\tilde{\mathcal{C}}_{b\ell}$, along with their associated amplitudes, where the shape operators are defined below:

\begin{equation}
\begin{aligned}
\tilde{\mathcal{C}}_{b \ell}^{T T}&=\sum_{\ell^{\prime}} K_{\ell \ell^{\prime}} F_{\ell^{\prime}}^{T T} B_{\ell^{\prime}}^{2} \mathcal{C}_{\ell^{\prime}}^{TT} \chi_{b}\left(\ell^{\prime}\right) \\
{}_\pm \tilde{\mathcal{C}}_{b \ell}^{EE}&=\sum_{\ell^{\prime}} {}_\pm K_{\ell \ell^{\prime}} F_{\ell^{\prime}}^{EE} B_{\ell^{\prime}}^{2} \mathcal{C}_{\ell^{\prime}}^{EE} \chi_{b}\left(\ell^{\prime}\right) \\
{}_\pm \tilde{\mathcal{C}}_{b \ell}^{BB}&=\sum_{\ell^{\prime}} {}_\pm K_{\ell \ell^{\prime}} F_{\ell^{\prime}}^{BB} B_{\ell^{\prime}}^{2} \mathcal{C}_{\ell^{\prime}}^{BB} \chi_{b}\left(\ell^{\prime}\right) \\
\tilde{\mathcal{C}}_{b \ell}^{TE}&=\sum_{\ell^{\prime}} {}_\times K_{\ell \ell^{\prime}} F_{\ell^{\prime}}^{TE} B_{\ell^{\prime}}^{2} \mathcal{C}_{\ell^{\prime}}^{TE} \chi_{b}\left(\ell^{\prime}\right) \\
\tilde{\mathcal{C}}_{b \ell}^{TB}&=\sum_{\ell^{\prime}} {}_\times K_{\ell \ell^{\prime}} F_{\ell^{\prime}}^{TB} B_{\ell^{\prime}}^{2} \mathcal{C}_{\ell^{\prime}}^{TB} \chi_{b}\left(\ell^{\prime}\right) \\
\tilde{\mathcal{C}}_{b \ell}^{EB}&=\sum_{\ell^{\prime}} ({}_+ K_{\ell \ell^{\prime}}-{}_- K_{\ell \ell^{\prime}}) F_{\ell^{\prime}}^{EB} B_{\ell^{\prime}}^{2} \mathcal{C}_{\ell^{\prime}}^{EB} \chi_{b}\left(\ell^{\prime}\right) \\
\end{aligned}
\label{eq:cbl}
\end{equation}

The shape operators, or "Cee-bee-ells" are simply understood to be the binned power we would expect given what we know of the coupling between our experiment and the sky. We have different shape expectations for the different signals we measure. Chiefly, the four that XFaster has currently implemented are CMB, dust, residual noise (that is, noise that is not accounted for in the noise simulation ensemble), and null signal. Each of these modifies the equation above somewhat, and we'll go into more detail about that further on in this guide.

The signal component of the covariance can then be written as

\begin{equation}
\tilde{\mathbf{S}}_\ell=
\begin{bmatrix}
\sum_b q_b^{TT}\tilde{\mathcal{C}}_{b\ell}^{TT} & \sum_b q_b^{TE}\tilde{\mathcal{C}}_{b\ell}^{TE} & \sum_b q_b^{TB}\tilde{\mathcal{C}}_{b\ell}^{TB} \\ 
-- & \sum_b q_b^{EE} {}_+\tilde{\mathcal{C}}_{b\ell}^{EE}+ \sum_b q_b^{BB} {}_-\tilde{\mathcal{C}}_{b\ell}^{BB} & \sum_b q_b^{EB}\tilde{\mathcal{C}}_{b\ell}^{EB} \\ 
-- & -- & \sum_b q_b^{BB} {}_+\tilde{\mathcal{C}}_{b\ell}^{BB}+ \sum_b q_b^{EE} {}_-\tilde{\mathcal{C}}_{b\ell}^{EE} \\ 
\end{bmatrix}
\label{eq:signal}
\end{equation}

To construct equations \ref{eq:qb} and \ref{eq:fisher}, we need to take the derivatives of equation \ref{eq:signal} with respect to each $q_b$. It's straightforward to read off the derivative terms:

\begin{align}
\frac{\partial \tilde{\mathbf{S}}_{\ell}}{\partial q_{b}^{TT}} &= 
   \begin{bmatrix}
   \tilde{\mathcal{C}}_{b\ell}^{TT} & 0 & 0 \\ 0 & 0 & 0 \\ 0 & 0 & 0 \\
   \end{bmatrix}
&
\frac{\partial \tilde{\mathbf{S}}_{\ell}}{\partial q_{b}^{TE}} &= 
   \begin{bmatrix}
   0 & \tilde{\mathcal{C}}_{b\ell}^{TE} & 0 \\ \tilde{\mathcal{C}}_{b\ell}^{TE} & 0 & 0 \\ 0 & 0 & 0 \\
   \end{bmatrix} 
\nonumber
\\
\nonumber
\\
\frac{\partial \tilde{\mathbf{S}}_{\ell}}{\partial q_{b}^{EE}} &= 
   \begin{bmatrix}
   0 & 0 & 0 \\ 0 & {}_+\tilde{\mathcal{C}}_{b\ell}^{EE} & 0 \\ 0 & 0 & {}_-\tilde{\mathcal{C}}_{b\ell}^{EE} \\
   \end{bmatrix}
&
\frac{\partial \tilde{\mathbf{S}}_{\ell}}{\partial q_{b}^{BB}} &= 
   \begin{bmatrix}
   0 & 0 & 0 \\ 0 & {}_-\tilde{\mathcal{C}}_{b\ell}^{BB} & 0 \\ 0 & 0 & {}_+\tilde{\mathcal{C}}_{b\ell}^{BB} \\
   \end{bmatrix}
\label{eq:dsdqb}
\\ 
\nonumber
\\
\frac{\partial \tilde{\mathbf{S}}_{\ell}}{\partial q_{b}^{TB}} &= 
   \begin{bmatrix}
   0 & 0 & \tilde{\mathcal{C}}_{b\ell}^{TB} \\ 0 & 0 & 0 \\ \tilde{\mathcal{C}}_{b\ell}^{TB} & 0 & 0 \\
   \end{bmatrix}
&
\frac{\partial \tilde{\mathbf{S}}_{\ell}}{\partial q_{b}^{EB}} &= 
   \begin{bmatrix}
   0 & 0 & 0 \\ 0 & 0 & \tilde{\mathcal{C}}_{b\ell}^{EB} \\ 0 & \tilde{\mathcal{C}}_{b\ell}^{EB} & 0 \\
   \end{bmatrix} 
\nonumber
\\
\end{align}

So now everything is set up that we need, and we just need to build the ingredients. The rest of this document will be how we get each of the terms, but the main engine of XFaster, once it has all the ingredients, is to iterate on equations \ref{eq:qb} and \ref{eq:fisher}. So,
1. Start with an initial guess at the $q_b$s, which we set to be 1.
2. Compute the Fisher matrix with Equation \ref{eq:fisher}.
3. Plug that into Equation \ref{eq:qb} to get a new guess for $q_b$.
4. Repeat until some convergence criterion is met.

We can use all these same tools to also fit for the transfer function-- instead of using $\tilde{\mathbf{D}}_\ell^{obs}-\tilde{\mathbf{N}}_\ell$ for our measured signal spectrum, we just use the ensemble average of the signal simulations, and set $F_\ell$ in Equation \ref{eq:cbl} to be 1. Then, the $q_b$s that pop out are just the transfer function itelf, and the inverse Fisher matrix gives the error on the transfer function.

## The main ingredients of the XFaster code

Before we get into the functions that produce each of the components that gets fed into equations \ref{eq:qb} and \ref{eq:fisher}, let's first talk about how XFaster is structured and what it expects as inputs.

There are two main python modules in XFaster: `xfaster_exec.py` and `xfaster_class.py`. `xfaster_exec` was written to mirror `unimap_exec`. It contains two main functions: `xfaster_run` and `xfaster_submit`. `xfaster_run` calls all of the functions to make XFaster happen (all located in `xfaster_class`) in the order they need to happen. `xfaster_submit` takes arguments for submitting the job to a queue. XFaster is not highly parallelized like `unimap`. The only real benefit it can get from using multiple cores is through the OMP speed up from numpy matrix operations. Therefore, unless you're drowning in more cores than you know what to do with, I would recommend only using one core for XFaster jobs, and splitting things up as best you can in other ways (ie, if running sims, have a handful of sims per job instead of running them all in series). 

There are a few other modules you might interact with: 
* `xfaster_tools.py`: contains small functions used in xfaster_class
* `parse_tools.py`: contains a bunch of tools for converting between data structures, especially between dictionaries and matrices
* `output_class.py`: contains tools for parsing data on disk for post-processing. This will only be very useful if you're looking at XFaster outputs created prior to July 22, 2019, when things were stored as arrays instead of dictionaries. We would now recommend loading the outputs directly with `xfaster.load_and_parse('<output>.npz')` and grabbing things in the dictionaries there. This is illustrated in the last section of this document.

## Specifying what data to use

So the top level thing you interact with is `xfaster_exec`, which takes your arguments, and has some reasonable defaults for any you don't provide. What does it take other than arguments? Maps! It wants a bunch of signal and noise simulations, in addition to the data maps and masks. Rather than pointing to each map individually, we have a directory structure that the code expects. I've put the ensemble on labah, and its contents look like this:

In [ ]:
ls ../maps_example

Each of data, signal, noise, and mask has a top level directory with a preordained, fixed prefix (`data`, `signal`, `noise`, `mask`) and then some suffix specified by the user which is appended with an underscore. So, for example, to run XFaster on this set of maps, I need to specify in my arguments: `clean_type=raw` (this is a confusing name for a variable, but just remember this designates the data type), `signal_type=flatBBDl`, `noise_type=stationary`, `mask_type=pointsource_latlon_aphalfdeg`.

Masks have to be named a certain way within the directory (`mask_map_<tag>.fits`) and have to contain both an I and P mask if running XFaster in polarized mode (which you should always be doing). We have always used the same mask for I and P, but it's not required. You must have a mask file for each map tag, so if I'm doing a run with maps for 90 and 150 GHz maps, I need a mask for each of those, even if it's the same mask. You can use symlinks to save on disk space in that case.

To indicate which maps I want, I pass the argument `data_subset` a glob-parseable path relative to the top level data directory-- in this case, `data_raw`. [Glob](https://docs.python.org/3/library/glob.html) works just like the unix shell does for matching file paths, so it is easy to test in advance which maps you're going to get. Say I want to do a 150 GHz spectrum using the quarter chunks. I'm going to test this first!

In [ ]:
ls ../maps_example/data_raw/*150* #nope

In [ ]:
ls ../maps_example/data_raw/*/*150* #yep-- data_subset='*/*150*' 

Or instead, if I want to run over all the maps, I would do

In [ ]:
ls ../maps_example/data_raw/*/* # data_subset='*/*'

This is all implemented in `_get_files` like so:
```python
# find all map files
map_root = os.path.join(data_root, 'data_{}'.format(clean_type))
map_files = []
data_subset = data_subset.split(",")
for f in np.atleast_1d(data_subset):
    map_files.extend(glob.glob(os.path.join(map_root,
                                            '{}.fits'.format(f))))
map_files = sorted(map_files)
map_files = [f for f in map_files if
             os.path.basename(f).startswith('map_')]
map_tags = [os.path.splitext(os.path.basename(f))[0].split('_', 1)[1]
            for f in map_files]
```
So you've specified which data maps you want to compute power spectra for. A reasonable question would be, does that globbable expression then just get applied to the signal and noise directory to get signal and noise maps? If it did, that would mean my first expression for the 150 GHz maps would also get every 150th sim. Instead, XFaster uses whatever matches it found in data, and tries to match those to maps in the sims directory, with the only difference the sim index that's appended. Here's what that looks like for signal sims, in the same function:
```python
# find all corresponding signal sims
signal_root = os.path.join(data_root, 'signal_{}'.format(signal_type))
num_signal = None
signal_files = []
for f in map_files:
    sfiles = sorted(glob.glob(
        f.replace(map_root, signal_root).replace(
            '.fits', '_{}.fits'.format(signal_subset))))
    nsims1 = len(sfiles)
    if not nsims1:
        raise OSError('Missing signal sims for {}'.format(f))
    num_signal = nsims1
    signal_files.append(sfiles)
```

It's easy to get errors at the reading files step. Add log messages around here to see what's going wrong. XFaster requires that all your maps have the same number of sims, though you are allowed to have a different from of signal and noise sims. In the labah data set, I have 1000 signal and noise sims for SPIDER, but only 200 of each for Planck. So I need to tell XFaster to only use the first 200 signal and noise sims it finds. I can do that using the `signal_subset` and `noise_subset` arguments, which default to `'*'`. Now, I need to specify (in a glob-parseable manner) which tags that come after `map_90_`, eg, to use. Each sim is indexed with four digits, so `map_90_0000.fits, map_90_0001.fits`, etc. If I want the first 200 of each sim, I'm first going to test my expression (I'm piping it into a line count to reduce the output, but you don't have to):

In [ ]:
ls ../maps_example/signal_synfast/full/map_95_0[0-1] | wc -l # this won't work

In [ ]:
ls ../maps_example/signal_synfast/full/map_95_0[0-1]* | wc -l # yep! signal_subset='0[0-1]*';

## Step by step through the functions called in xfaster_exec

Now we'll proceed to stepping through each function in xfaster_exec. You'll never run the code this way, you'll just call xfaster_run or xfaster_submit with the arguments that then get passed to these functions. But we'll do it this way so we can illustrate some of the intermediate data products as we go.

I'll do a run with all six SPIDER and Planck frequencies. First, we'll import XFaster and initialize our XFaster class with some arguments.

In [ ]:
import xfaster as xf
import matplotlib.pyplot as plt
from matplotlib import colors
import numpy as np

In [ ]:
X = xf.XFaster(config='../config_example.ini', output_root='../outputs_example')